<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_DMM/blob/main/LSTM_Conv2D_tf2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)


#Data Load
df_train = pd.read_csv('/dir')
df_test = pd.read_csv('/dir')
train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]
print(train_data)
print(test_data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back

In [ ]:
#Hyper parameters
seqLength = 10
dataDim = df_test.shape[1]
outputDim = 1
hiddenDim = 15
lr = 0.001
iterations = 20
batch_size = 16

trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim-1])
testSet_min = (testSet_min[dataDim-1])


trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

print(trainSet.shape) # (5807,5)
print(testSet.shape) # (10882,5)

In [ ]:
#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length): 
        print(i)
        _x = time_series.iloc[i:i + seq_length, :  ]
        _y = time_series.iloc[i + seq_length, :  ] 
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(trainSet, seqLength)
testX, testY = sliding_window(testSet, seqLength)


In [ ]:
# Stacked LSTM and Conv2D Mixed Modeling
# Return Sequences is very important for stacking
# This means weather output is Sequence of this layer or all of sequences
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

filters = 4
kernel_size = (2,2)
pooling_size = (2,2)
stride = (1,1)
initializer = tf.keras.initializers.GlorotNormal()
input_dim = trainSet.shape[1]
input_shape = train_data.shape[1]
outputDim = 1
print(input_dim)

model = keras.Sequential()
model.add(layers.LSTM(units = hiddenDim, activation='tanh',input_shape = (seqLength,input_dim), return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'tanh',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'tanh',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'tanh',return_sequences = True, kernel_initializer = initializer))
model.add(layers.Reshape((seqLength,hiddenDim, 1)))
model.add(Conv2D(filters = filters, kernel_size = kernel_size, strides = stride, activation = 'relu', input_shape = (seqLength,hiddenDim)))
model.add(Conv2D(filters = filters, kernel_size = kernel_size, strides = stride, activation = 'relu',))
model.add(Conv2D(filters = filters, kernel_size = kernel_size, strides = stride, activation = 'relu',))
model.add(Conv2D(filters = filters, kernel_size = kernel_size, strides = stride, activation = 'relu',))
model.add(Flatten())
model.add(layers.Dense(input_dim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')
model.summary()


In [ ]:
# Training
hist = model.fit(trainX, trainY, epochs = iterations, batch_size = batch_size)

In [ ]:
# Evaluate Testset
res = model.evaluate(testX, testY, batch_size = batch_size)
print(res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow


In [ ]:
# Estimating
xhat = testX
yhat = model.predict(xhat)
print(testY)
print(yhat)

print("Evaluate : {}".format(np.sqrt(np.average((yhat - testY)**2))))

In [ ]:
#Estimating
estimated = back_MinMax(yhat,testSet_max,testSet_min) 
estimated = estimated[:,-1]
real = testSet_for_result.iloc[seqLength:,-1] # estimated + seqLength

In [ ]:
plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by LSTM_Conv2D Deeplearning Mixture Model")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

print("Estimated : ",estimated.shape,estimated)
print("Real : ",real.shape, real)